In [12]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
!pip install tf-nightly==2.4.0.dev20200929

     |████████████████████████████████| 391.7MB 36kB/s 
     |████████████████████████████████| 20.1MB 50.8MB/s 
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: numpy 1.19.4
    Uninstalling numpy-1.19.4:
      Successfully uninstalled numpy-1.19.4
  Found existing installation: tf-nightly 2.5.0.dev20201105
    Uninstalling tf-nightly-2.5.0.dev20201105:
      Successfully uninstalled tf-nightly-2.5.0.dev20201105


In [1]:
!gsutil -q -m cp -r /content/drive/My\ Drive/SocialDistance/DPDMEfficientDetD0/fine_tuned_model /content/drive/My\ Drive/SocialDistance/DPDM_D0_tflite/

In [2]:
import tensorflow as tf
tf.__version__

'2.4.0-dev20200929'

In [4]:
%cd /content/drive/My\ Drive/SocialDistance/DPDM_D0_tflite/
%ls 'fine_tuned_model/saved_model'
saved_model_dir = 'fine_tuned_model/saved_model'


model = tf.saved_model.load(saved_model_dir)
model.signatures[tf.saved_model.DEFAULT_SERVING_SIGNATURE_DEF_KEY].inputs[0].set_shape([1, 512, 512, 3])
tf.saved_model.save(model, "saved_model_new", signatures=model.signatures[tf.saved_model.DEFAULT_SERVING_SIGNATURE_DEF_KEY])

/content/drive/My Drive/SocialDistance/DPDM_D0_tflite
saved_model.pb  variables/
INFO:tensorflow:Assets written to: saved_model_new/assets


In [5]:
# Convert
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir='saved_model_new', signature_keys=['serving_default'])
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]
tflite_model = converter.convert()

In [6]:
import numpy as np


## TFLite Interpreter to check input shape
interpreter = tf.lite.Interpreter(model_content=tflite_model)
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Print input and output tensors
print("INPUT DETAILS: \n",input_details)
print("OUTPUT DETAILS: \n",output_details)
# Test the model on random input data.
input_shape = input_details[0]['shape']
print(input_shape)
interpreter.invoke()
output_data = interpreter.get_tensor(output_details[0]['index'])
print(output_data)

INPUT DETAILS: 
 [{'name': 'serving_default_input_tensor:0', 'index': 0, 'shape': array([  1, 512, 512,   3], dtype=int32), 'shape_signature': array([  1, 512, 512,   3], dtype=int32), 'dtype': <class 'numpy.uint8'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
OUTPUT DETAILS: 
 [{'name': 'StatefulPartitionedCall:4', 'index': 66519, 'shape': array([1, 1], dtype=int32), 'shape_signature': array([ 1, -1], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}, {'name': 'StatefulPartitionedCall:6', 'index': 66380, 'shape': array([    1, 49104,     4], dtype=int32), 'shape_signature': array([    1, 49104,     4], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0

In [7]:
#tflite_file = '/content/fine_tuned_model/mask_detector.tflite'
tflite_file = '/content/drive/My Drive/SocialDistance/DPDM_D0_tflite/fine_tuned_model/DPDMD0_person_detector.tflite'
#%cd /drive/My\ Drive/Maithri_TF/fine_tuned_model/
%cd /content/drive/My\ Drive/SocialDistance/DPDMEfficientDetD0/fine_tuned_model/
#now save the tflite model to the file
#tflite_model.save(tflite_file)   #Note this does not seem to work although in google documentation
open(tflite_file, "wb").write(tflite_model)
!ls /content/drive/My\ Drive/SocialDistance/DPDM_D0_tflite/fine_tuned_model

/content/drive/My Drive/SocialDistance/DPDMEfficientDetD0/fine_tuned_model
checkpoint		       pipeline.config	valid_pipeline.config
DPDMD0_person_detector.tflite  saved_model


In [8]:
from google.colab import files
#files.download(tflite_model)

!ls /content/drive/My\ Drive/SocialDistance/DPDM_D0_tflite/fine_tuned_model

files.download('/content/drive/My Drive/SocialDistance/DPDM_D0_tflite/fine_tuned_model/DPDMD0_person_detector.tflite')

checkpoint		       pipeline.config	valid_pipeline.config
DPDMD0_person_detector.tflite  saved_model


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>